In [4]:
"""
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import serial
import wave

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024*8# samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

wav_file = wave.open(r"C:\Users\rajka\Downloads\Sunflower-(Spider-Man_-Into-the-Spider-Verse)(PagalWorld).wav", 'rb')
# Set up serial communication with Arduino
arduino = serial.Serial('COM4', 9600)
time.sleep(2)  # Wait for Arduino to initialize



In [5]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
# stream = p.open(
#     format=FORMAT,
#     channels=CHANNELS,
#     rate=RATE,
#     input=True,
#     output=True,
#     frames_per_buffer=CHUNK
# )
stream = p.open(
    format=p.get_format_from_width(wav_file.getsampwidth()),
    channels=wav_file.getnchannels(),
    rate=wav_file.getframerate(),
    input=False,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(0, 255)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')
chosen_frequencies = [250,500,750,2500,10000]
# Indices corresponding to chosen frequencies
chosen_indices = [np.abs(xf - freq).argmin() for freq in chosen_frequencies]
# for measuring frame rate
frame_count = 0
start_time = time.time()

# while True:
    
#     # binary data
#     data = stream.read(CHUNK)  
    
#     # convert data to integers, make np array, then offset it by 127
#     data_int = struct.unpack(str(2 * CHUNK) + 'B', data)

print('Playback started')

# Read the audio data from the .wav file
data = wav_file.readframes(CHUNK)

while len(data)>0:
    
    # convert data to integers, make np array, then offset it by 127
    data_int = np.array(struct.unpack(f"{2*CHUNK}h", data), dtype=np.int16) + 128

    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    # line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_int)
    # line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (128 * CHUNK))

    stream.write(data)
    # x=(np.abs(yf[0:CHUNK])  / (128 * CHUNK))
    # print(x.shape)
    # update figure canvas
    # for freq, value in zip(xf, np.abs(yf[0:CHUNK]) / (128 * CHUNK)):
    #     print(f"Frequency: {freq:.2f} Hz, Value: {value:.2f}")
    for freq, index in zip(chosen_frequencies, chosen_indices):
        value = np.abs(yf[index]) / (128 * CHUNK)
        print(f"Frequency: {freq} Hz, Value: {value}")
        message = f"{freq}:{value}\n".encode()  # Encode message as bytes
        arduino.write(message)
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

        # # Control frame rate
        # if frame_count % (40) == 0:
        #     elapsed_time = time.time() - start_time
        #     current_frame_rate = frame_count / elapsed_time

        # # Check if desired frame rate is achieved
        # if current_frame_rate >= desired_frame_rate:
        #     break
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break
     # Read the next chunk of audio data from the .wav file
    data = wav_file.readframes(CHUNK)

print('Playback finished')

stream started
Playback started
Frequency: 250 Hz, Value: 0.0
Frequency: 500 Hz, Value: 0.0
Frequency: 750 Hz, Value: 0.0
Frequency: 2500 Hz, Value: 0.0
Frequency: 10000 Hz, Value: 0.0
Frequency: 250 Hz, Value: 0.0
Frequency: 500 Hz, Value: 0.0
Frequency: 750 Hz, Value: 0.0
Frequency: 2500 Hz, Value: 0.0
Frequency: 10000 Hz, Value: 0.0
Frequency: 250 Hz, Value: 0.07378264818848469
Frequency: 500 Hz, Value: 0.008843233871565152
Frequency: 750 Hz, Value: 0.004740090859740379
Frequency: 2500 Hz, Value: 0.0018369249972483995
Frequency: 10000 Hz, Value: 0.0002902381864468188
Frequency: 250 Hz, Value: 0.3552923395510583
Frequency: 500 Hz, Value: 0.04300733863522998
Frequency: 750 Hz, Value: 0.05993428889439833
Frequency: 2500 Hz, Value: 0.00748032821539304
Frequency: 10000 Hz, Value: 0.00039640084050514995
Frequency: 250 Hz, Value: 0.253436434889093
Frequency: 500 Hz, Value: 0.010493566083820453
Frequency: 750 Hz, Value: 0.07253530709298248
Frequency: 2500 Hz, Value: 0.00432593403262076
Freq

: 

: 

Playback started
Frequency: 100 Hz, Value: 0.0
Frequency: 250 Hz, Value: 0.0
Frequency: 500 Hz, Value: 0.0
Frequency: 750 Hz, Value: 0.0
Frequency: 1000 Hz, Value: 0.0
Frequency: 2500 Hz, Value: 0.0
Frequency: 100 Hz, Value: 4.098153724698867e-07
Frequency: 250 Hz, Value: 3.5359581051632348e-06
Frequency: 500 Hz, Value: 4.177976174144751e-07
Frequency: 750 Hz, Value: 6.209444772317282e-07
Frequency: 1000 Hz, Value: 1.266634112322634e-07
Frequency: 2500 Hz, Value: 4.7031808335267935e-08
Frequency: 100 Hz, Value: 1.4123002527404484e-05
Frequency: 250 Hz, Value: 3.7189254087641045e-06
Frequency: 500 Hz, Value: 1.7615981007931058e-06
Frequency: 750 Hz, Value: 5.850292499793214e-07
Frequency: 1000 Hz, Value: 1.0051465114451429e-06
Frequency: 2500 Hz, Value: 2.0880494926390643e-07
Frequency: 100 Hz, Value: 3.8393003633389864e-05
Frequency: 250 Hz, Value: 1.635086175058244e-05
Frequency: 500 Hz, Value: 1.628741014997693e-05
Frequency: 750 Hz, Value: 4.6562736917656765e-06
Frequency: 1000 Hz, 

: 

: 